In [1]:
#!/usr/bin/env python2
import os
import json
import re
import scipy.io as spio
import numpy as np
from json_tricks.np import dump, dumps, load, loads
import pandas as pd

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]


In [2]:
source = '/nas/volume1/2photon/projects'
experiment = 'gratings_phaseMod'
session = '20171009_CE059'
acquisition = 'FOV1_zoom3x'
functional_dir = 'functional'

acquisition_dir = os.path.join(source, experiment, session, acquisition)
figdir = os.path.join(acquisition_dir, 'example_figures')

# # ================================================================================
# # frame info:
# # ================================================================================
# first_frame_on = 50
# stim_on_sec = 0.5
# iti = 1.
# vols_per_trial = 15
# same_order = True
# # =================================================================================

In [3]:
# Load reference info:
ref_json = 'reference_%s.json' % functional_dir 
with open(os.path.join(acquisition_dir, ref_json), 'r') as fr:
    ref = json.load(fr)

# Load SI meta data:
si_basepath = ref['raw_simeta_path'][0:-4]
simeta_json_path = '%s.json' % si_basepath
with open(simeta_json_path, 'r') as fs:
    simeta = json.load(fs)

file_names = sorted([k for k in simeta.keys() if 'File' in k], key=natural_keys)
nfiles = len(file_names)

In [4]:
volumerate = float(simeta['File001']['SI']['hRoiManager']['scanVolumeRate'])
framerate = float(simeta['File001']['SI']['hRoiManager']['scanFrameRate'])
nvolumes = int(simeta['File001']['SI']['hFastZ']['numVolumes'])

print "N volumes:", nvolumes
print "Volume Rate (Hz):", volumerate
print "Frame Rate (Hz):", framerate

N volumes: 6620
Volume Rate (Hz): 44.7027
Frame Rate (Hz): 44.7027


In [5]:
# Get PARADIGM INFO:
path_to_functional = os.path.join(acquisition_dir, functional_dir)
paradigm_dir = 'paradigm_files'
path_to_paradigm_files = os.path.join(path_to_functional, paradigm_dir)

# Get stim-order files:
serialdata_fns = os.listdir(path_to_paradigm_files)
serialdata_fns = sorted([f for f in serialdata_fns if 'serial_data' in f])
print "Found %i serial-data files, and %i TIFFs." % (len(serialdata_fns), nfiles)

Found 10 serial-data files, and 10 TIFFs.


In [107]:
fid = 2
print "fid:", fid
print "fn:", serialdata_fns[fid]
ardata = pd.read_csv(os.path.join(path_to_paradigm_files, serialdata_fns[fid]), sep='\t')

fid: 2
fn: 20171009_CE059_FOV1_zoom3x_003_serial_data_20171009170545679404.txt


In [108]:
ardata.columns

Index([u'acquisition_trigger', u' frame_trigger', u' pixel_clock',
       u' abosolute_arduino_time', u' relative_arduino_time',
       u'     absolute_computer_time', u' relative_computer_time'],
      dtype='object')

In [109]:
frame_triggers = ardata[' frame_trigger']
print len(frame_triggers)

arduino_time = ardata[' relative_arduino_time']
computer_time = ardata[' relative_computer_time']

148436


In [110]:
bitcodes = ardata[' pixel_clock']

### TEST MW files:

In [11]:
mw_fns = os.listdir(path_to_paradigm_files)
mw_fns = sorted([f for f in mw_fns if f.endswith('.mwk')])
mw_fns

['20171009_CE059_FOV1_zoom3x_001.mwk',
 '20171009_CE059_FOV1_zoom3x_002.mwk',
 '20171009_CE059_FOV1_zoom3x_003.mwk',
 '20171009_CE059_FOV1_zoom3x_004.mwk',
 '20171009_CE059_FOV1_zoom3x_005.mwk',
 '20171009_CE059_FOV1_zoom3x_006.mwk',
 '20171009_CE059_FOV1_zoom3x_007.mwk',
 '20171009_CE059_FOV1_zoom3x_008.mwk',
 '20171009_CE059_FOV1_zoom3x_009.mwk',
 '20171009_CE059_FOV1_zoom3x_010.mwk']

In [12]:
import pymworks
dfn = os.path.join(path_to_paradigm_files, mw_fns[fid])
df = pymworks.open(os.path.join(path_to_paradigm_files, mw_fns[fid]))
print mw_fns[fid]
#pixelevents, stimevents, trialevents, trigger_times, sprint ession_info = get_stimulus_events(curr_dfn, stimtype=stimtype, phasemod=phasemod, triggername=trigger_varname)




20171009_CE059_FOV1_zoom3x_001.mwk


In [17]:
curr_dfn = dfn
stimtype = 'grating'
phasemod = True
trigger_varname = 'frame_trigger'
pixelevents, stimevents, trialevents, trigger_times, session_info = get_stimulus_events(curr_dfn, stimtype=stimtype, phasemod=phasemod, triggername=trigger_varname)


****************************************************************
Parsing file
/nas/volume1/2photon/projects/gratings_phaseMod/20171009_CE059/FOV1_zoom3x/functional/paradigm_files/20171009_CE059_FOV1_zoom3x_001.mwk... 
Found 1 start events in session.
Bounds:  [[1218328934002, 1218484705733]]
bound ID: 0 155.771731 sec
****************************************************************
[[1218328934002, 1218484705733]]
................................................................
SECTION 0
................................................................
first SI-trigger ON event received:  Event[code=44, name=frame_trigger, time=1218336288001, value=0]
first SI-trigger OFF event received:  Event[code=44, name=frame_trigger, time=1218484296161, value=1]
Duration of first run: 148.0082 sec.
Incrementally searching to find each pair of ON/OFF trigger events...
........................................................................................
Found 2 chunks from frame-on/-off triggers:

In [19]:
    # For EACH boundary found for a given datafile (dfn), make sure all the events are concatenated together:
    print "================================================================"
    print "MW parsing summary:"
    print "================================================================"
    if len(pixelevents) > 1:
        pixelevents = [item for sublist in pixelevents for item in sublist]
        pixelevents = list(set(pixelevents))
        pixelevents.sort(key=operator.itemgetter(1))
    else:
        pixelevents = pixelevents[0]
    print "Found %i pixel clock events." % len(pixelevents)

    # Check that all possible pixel vals are used (otherwise, pix-clock may be missing input):
    n_codes = set([i.value[-1]['bit_code'] for i in pixelevents])
    if len(n_codes)<16:
        print "Check pixel clock -- missing bit values..."


    if len(stimevents) > 1:
        stimevents = [item for sublist in stimevents for item in sublist]
        stimevents = list(set(stimevents))
        stimevents.sort(key=operator.itemgetter(1))
    else:
        stimevents = stimevents[0]
    print "Found %i stimulus on events." % len(stimevents)
    
    if not stimtype=='bar':
	if len(trialevents) > 1:
	    trialevents = [item for sublist in trialevents for item in sublist]
	    trialevents = list(set(trialevents))
	    trialevents.sort(key=operator.itemgetter(1))
	else:
	    trialevents = trialevents[0]
	print "Found %i trial epoch (stim ON + ITI) events." % len(trialevents)


    if len(trigger_times) > 1:
        trigger_times = [item for sublist in trigger_times for item in sublist]
        trigger_times = list(set(trigger_times))
        trigger_times.sort(key=operator.itemgetter(1))
    else:
        trigger_times = trigger_times[0]
    print "Found %i runs (i.e., trigger boundary events)." % len(trigger_times)

    if len(session_info) > 1:
        session_info = [item for sublist in session_info for item in sublist]
        session_info = list(set(trigger_times))
        session_info.sort(key=operator.itemgetter(1))
    else:
        session_info = session_info[0]
    #print "N sessions in file: .", len(session_info)
    print session_info

MW parsing summary:
Found 2906 pixel clock events.
Found 24 stimulus on events.
Found 49 trial epoch (stim ON + ITI) events.
Found 1 runs (i.e., trigger boundary events).
{'tboundary': [1218328934002, 1218484705733], 'stimduration': 2000, 'stimsize': 100, 'ITI': 4000}


In [25]:
    print "Found %i trials, corresponding to %i TIFFs." % (len(stimevents), len(trigger_times))
    refresh_rate = 60

Found 24 trials, corresponding to 1 TIFFs.


In [28]:

def get_timekey(item):
    return item.time


In [29]:

ntrials = len(stimevents)
post_itis = sorted(trialevents[2::2], key=get_timekey) # get_stimulus_events() returns alternating list of [trial1-stimonset, trial2-iti, trial2-stimonset, trial2-iti, etc...]


In [38]:

# Get dynamic-grating bicode events:
dynamic_stim_bitcodes = []
bitcodes_by_trial = dict((i+1, dict()) for i in range(len(stimevents)))
#for stim,iti in zip(sorted(stimevents, key=get_timekey), sorted(itis, key=get_timekey)):
for trialidx,(stim,iti) in enumerate(zip(sorted(stimevents, key=get_timekey), sorted(post_itis, key=get_timekey))):
    trialnum = trialidx + 1
    # For each trial, store all associated stimulus-bitcode events (including the 1st stim-onset) as a list of
    # display-update events related to that trial:
    current_bitcode_evs = [p for p in sorted(pixelevents, key=get_timekey) if p.time>=stim.time and p.time<=iti.time] # p.time<=iti.time to get bit-code for post-stimulus ITI
    current_bitcode_values = [p.value[-1]['bit_code'] for p in sorted(current_bitcode_evs, key=get_timekey)]
    dynamic_stim_bitcodes.append(current_bitcode_evs)
    bitcodes_by_trial[trialnum] = current_bitcode_values #current_bitcode_evs

# Roughly calculate how many pixel-clock events there should be. For static images, there should be 1 bitcode-event per trial.
# For drifting gratings, on a 60Hz monitor, there should be 60-61 bitcode-events per trial.
nexpected_pixelevents = (ntrials * (session_info['stimduration']/1E3) * refresh_rate) + ntrials + 1
nbitcode_events = sum([len(tr) for tr in dynamic_stim_bitcodes]) + 1 #len(itis) + 1 # Add an extra ITI for blank before first stimulus



In [39]:
print "Expected %i pixel events, missing %i pevs." % (nexpected_pixelevents, nexpected_pixelevents-nbitcode_events)


Expected 2905 pixel events, missing 0 pevs.


In [41]:
# GET TRIAL INFO FOR DB:
trial = dict((i+1, dict()) for i in range(len(stimevents)))
stimevents = sorted(stimevents, key=get_timekey)
trialevents = sorted(trialevents, key=get_timekey)
run_start_time = trialevents[0].time
for trialidx,(stim,iti) in enumerate(zip(sorted(stimevents, key=get_timekey), sorted(post_itis, key=get_timekey))):
    trialnum = trialidx + 1
    # blankidx = trialidx*2 + 1
    trial[trialnum]['start_time_ms'] = round(stim.time/1E3)
    trial[trialnum]['end_time_ms'] = round((iti.time + session_info['ITI']))
    if stimtype=='grating':
        ori = stim.value[1]['rotation']
        sf = round(stim.value[1]['frequency'], 2)
        stimname = 'grating-ori-%i-sf-%f' % (ori, sf)
        stimpos = [stim.value[1]['xoffset'], stim.value[1]['yoffset']]
    else:
        # TODO:  fill this out with the appropriate variable tags for RSVP images
        stimname = ''
        stimpos = ''
    stimsize = stim.value[1]['height']
    trial[trialnum]['stimuli'] = {'stimulus': stimname, 'position': stimpos, 'scale': stimsize}
    trial[trialnum]['stim_on_times'] = round((stim.time - run_start_time)/1E3)
    trial[trialnum]['stim_off_times'] = round((iti.time - run_start_time)/1E3)
    trial[trialnum]['all_bitcodes'] = bitcodes_by_trial[trialnum]
    #if stim.value[-1]['name']=='pixel clock':
    trial[trialnum]['stim_bitcode'] = stim.value[-1]['bit_code']
    trial[trialnum]['iti_bitcode'] = iti.value[-1]['bit_code']


In [44]:
trial[9]

{'all_bitcodes': [10,
  9,
  10,
  8,
  1,
  4,
  13,
  0,
  10,
  5,
  12,
  15,
  6,
  8,
  9,
  11,
  5,
  14,
  13,
  15,
  1,
  0,
  3,
  11,
  1,
  4,
  15,
  1,
  14,
  12,
  8,
  5,
  8,
  11,
  0,
  9,
  2,
  7,
  4,
  5,
  2,
  9,
  8,
  12,
  9,
  0,
  11,
  0,
  4,
  1,
  7,
  11,
  13,
  11,
  3,
  13,
  6,
  7,
  5,
  0,
  8,
  13,
  2,
  15,
  12,
  7,
  11,
  12,
  0,
  7,
  8,
  7,
  13,
  11,
  4,
  1,
  15,
  2,
  10,
  14,
  9,
  11,
  7,
  12,
  4,
  11,
  4,
  14,
  1,
  0,
  3,
  5,
  2,
  11,
  14,
  3,
  11,
  9,
  4,
  13,
  4,
  2,
  12,
  11,
  10,
  4,
  2,
  10,
  3,
  2,
  5,
  6,
  14,
  4,
  5,
  10,
  11,
  1,
  0,
  6,
  7],
 'end_time_ms': 1218390470903.0,
 'iti_bitcode': 7,
 'start_time_ms': 1218388467.0,
 'stim_bitcode': 10,
 'stim_off_times': 54151.0,
 'stim_on_times': 52151.0,
 'stimuli': {'position': [0, 0],
  'scale': 100,
  'stimulus': 'grating-ori-0-sf-0.100000'}}

In [650]:
from bokeh.charts import Histogram
from bokeh.charts import defaults,show, output_file
from bokeh.io import output_notebook
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [651]:
p1 = figure(title="Intervals between writes (ms)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

#medintervals = np.array([i for i in intervals if i<=10000000])

hist, edges = np.histogram(np.diff(arduino_time), density=True, bins=100)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

In [111]:
# frame_off_idxs = [idx for idx,val in enumerate(frame_triggers) if val==0]
frame_on_idxs = [idx+1 for idx,diff in enumerate(np.diff(frame_triggers)) if diff==1]
#print len(frame_on_idxs)
frame_on_idxs.append(0)
frame_on_idxs = sorted(frame_on_idxs)
print len(frame_on_idxs)


6620


In [112]:
frame_on_times = [arduino_time[i] for i in frame_on_idxs]
print "Time expected between frames (ms):", (1/framerate)*1000
print "N frames: ", len(frame_on_times)

Time expected between frames (ms): 22.3700134444
N frames:  6620


In [113]:
### Get bitcodes for each frame:
frame_bitcodes = dict()
for idx,frameidx in enumerate(frame_on_idxs):
    #framenum = 'frame'+str(idx)
    if idx==len(frame_on_idxs)-1:
        bcodes = bitcodes[frameidx:]
    else:
        bcodes = bitcodes[frameidx:frame_on_idxs[idx+1]]
    frame_bitcodes[idx] = bcodes

In [114]:
frame_bitcodes[100]

2236    10
2237    10
2238    10
2239    10
2240    10
2241    10
2242    10
2243    10
2244    10
2245    10
2246    10
2247    10
2248    10
2249    10
2250    10
2251    10
2252    10
2253    10
2254    10
2255    10
2256    10
2257    10
Name:  pixel_clock, dtype: int64

In [115]:
len(frame_bitcodes)

6620

In [116]:
frames_with_stim = [f for f in frame_bitcodes.keys() if sum(frame_bitcodes[f])>0]
frames_with_stim = sorted(frames_with_stim) #, key=natural_keys)
frames_with_stim[0:20]
# blank_frames = [k for k in frame_bitcodes.keys() if k < frames_with_stim[0]]
# print blank_frames

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [117]:
# for b in blank_frames:
#     frame_bitcodes.pop(b, None)

In [118]:
from collections import Counter
modes_by_frame = dict()
for frame in frame_bitcodes.keys():
    bitcode_counts = Counter(frame_bitcodes[frame])
    modes_by_frame[frame] = bitcode_counts.most_common(1)[0][0]

first_stim_frame = [k for k in sorted(modes_by_frame.keys()) if modes_by_frame[k]>0][0]
print "First real stim on frame ", first_stim_frame

blank_frames = [k for k in modes_by_frame.keys() if k < first_stim_frame]
print blank_frames
for b in blank_frames:
    modes_by_frame.pop(b, None)

First real stim on frame  4
[0, 1, 2, 3]


In [119]:
frame_bitcodes[3]

68     0
69     0
70     0
71     0
72     0
73     0
74     0
75     0
76     0
77     0
78     0
79     0
80     0
81     0
82     0
83     0
84     0
85     0
86     0
87    10
88    10
89    10
Name:  pixel_clock, dtype: int64

### Check parse_mw_trials.py output:

In [120]:
pydict_jsons = os.listdir(path_to_paradigm_files)
pydict_jsons = sorted([p for p in pydict_jsons if p.endswith('.json') and 'trial_info' in p], key=natural_keys)
pydict_jsons

['trial_info_20171009_CE059_FOV1_zoom3x_001.json',
 'trial_info_20171009_CE059_FOV1_zoom3x_002.json',
 'trial_info_20171009_CE059_FOV1_zoom3x_003.json',
 'trial_info_20171009_CE059_FOV1_zoom3x_004.json',
 'trial_info_20171009_CE059_FOV1_zoom3x_005.json',
 'trial_info_20171009_CE059_FOV1_zoom3x_006.json',
 'trial_info_20171009_CE059_FOV1_zoom3x_007.json',
 'trial_info_20171009_CE059_FOV1_zoom3x_008.json',
 'trial_info_20171009_CE059_FOV1_zoom3x_009.json',
 'trial_info_20171009_CE059_FOV1_zoom3x_010.json']

In [121]:
#fid = 0
print pydict_jsons[fid]
with open(os.path.join(path_to_paradigm_files, pydict_jsons[fid]), 'r') as f:
    trials = json.load(f)
print trials.keys()

trial_info_20171009_CE059_FOV1_zoom3x_003.json
[u'24', u'20', u'21', u'22', u'23', u'1', u'3', u'2', u'5', u'4', u'7', u'6', u'9', u'8', u'11', u'10', u'13', u'12', u'15', u'14', u'17', u'16', u'19', u'18']


In [122]:
trials['18']

{u'all_bitcodes': [4,
  0,
  5,
  8,
  4,
  5,
  8,
  13,
  6,
  1,
  6,
  11,
  14,
  7,
  14,
  6,
  12,
  0,
  10,
  11,
  6,
  11,
  13,
  5,
  14,
  0,
  12,
  13,
  1,
  13,
  15,
  9,
  2,
  14,
  2,
  7,
  3,
  2,
  13,
  10,
  0,
  3,
  9,
  15,
  13,
  12,
  9,
  4,
  12,
  8,
  0,
  14,
  5,
  8,
  9,
  5,
  3,
  10,
  2,
  13,
  9,
  5,
  7,
  11,
  13,
  8,
  3,
  12,
  4,
  6,
  1,
  9,
  12,
  9,
  1,
  12,
  3,
  6,
  14,
  1,
  10,
  4,
  8,
  11,
  7,
  10,
  15,
  7,
  9,
  7,
  2,
  12,
  8,
  9,
  1,
  7,
  3,
  15,
  2,
  10,
  14,
  9,
  6,
  7,
  10,
  7,
  5,
  10,
  11,
  7,
  9,
  2,
  5,
  0,
  3,
  13,
  5,
  4,
  15,
  6,
  1],
 u'end_time_ms': 1218924799445.0,
 u'iti_bitcode': 1,
 u'start_time_ms': 1218922795.0,
 u'stim_bitcode': 4,
 u'stim_off_times': 108302.0,
 u'stim_on_times': 106302.0,
 u'stimuli': {u'position': [0, 0],
  u'scale': 100,
  u'stimulus': u'grating-ori-120-sf-0.100000'}}

In [136]:
modes_by_frame = dict()
for frame in frame_bitcodes.keys():
    bitcode_counts = Counter(frame_bitcodes[frame])
    modes_by_frame[frame] = bitcode_counts.most_common(1)[0][0]

first_stim_frame = [k for k in sorted(modes_by_frame.keys()) if modes_by_frame[k]>0][0]
print "First real stim on frame ", first_stim_frame

First real stim on frame  4


In [137]:
modes = [(idx, modes_by_frame[f]) for idx,f in enumerate(sorted(modes_by_frame.keys()))]
find_diffs = [idx+1 for idx,val in enumerate(np.diff([m[1] for m in modes])) if not val==0]

In [138]:
changeframes = [modes[idx] for idx in find_diffs]
print changeframes[0:5]

[(4, 10), (184, 14), (185, 1), (186, 12), (187, 7)]


In [140]:
trialdict = dict()
allframes = sorted(frame_bitcodes.keys()) #, key=natural_keys)
curr_frames = sorted(allframes[first_stim_frame+1:]) #, key=natural_keys)
#curr_frames = sorted(allframes[3142:])

for trial in ['1']: #sorted(trials.keys(), key=natural_keys): #sorted(trials.keys, key=natural_keys):
    #print trial
    trialdict[trial] = dict()
    trialdict[trial]['name'] = trials[trial]['stimuli']['stimulus']
    trialdict[trial]['duration'] = trials[trial]['stim_off_times'] - trials[trial]['stim_on_times']

    first_found_frame = []
    minframes = 5
    for bitcode in trials[trial]['all_bitcodes']:
        #print bitcode # = 2
            looking = True
            while looking is True:
                #for idx,frame in enumerate(sorted(curr_frames)):
                for frame in sorted(curr_frames):
#                     if frame > first_stim_frame:
                    tmp_frames = [i for i in frame_bitcodes[frame] if i==bitcode]
                    consecutives = [i for i in np.diff(tmp_frames) if i==0]

                    if frame>1:
                        tmp_frames_pre = [i for i in frame_bitcodes[int(frame)-1] if i==bitcode]
                        consecutives_pre = [i for i in np.diff(tmp_frames_pre) if i==0]

                    if frame>1 and len(consecutives_pre)>=minframes:
                        #first_frame = frame - 1
                        if len(consecutives_pre) > len(consecutives):
                            first_frame = int(frame) - 1
                        elif len(consecutives)>=minframes:
                            first_frame = int(frame)
                        print "found2...", bitcode, first_frame #len(curr_frames)
                        looking = False

#                     if frame>1 and len(consecutives_pre)>=minframes and len(consecutives)>=minframes:
#                         if len(consecutives_pre) > len(consecutives):
#                             first_frame = int(frame) - 1
#                         else:
#                             first_frame = frame
                    elif len(consecutives)>=minframes:
                        first_frame = frame 
                        print "found...", bitcode, first_frame #len(curr_frames)
                        looking = False

                    if looking is False:
                        break

            first_found_frame.append((bitcode, first_frame)) #first_frame))
            curr_frames = allframes[first_frame+1:] #curr_frames[idx:] #curr_frames[first_frame:]

    print "Trial %i dur (s):" % int(trial)
    print (first_found_frame[-1][1] - first_found_frame[0][1])/framerate

    trialdict[trial]['stim_on_idx'] = first_found_frame[0][1]
    trialdict[trial]['stim_off_idx'] = first_found_frame[-1][1]
    
    #print first_found_frame[-1][1] - first_found_frame[0][1]

#
                    

found... 0 183
found... 14 184
found... 1 185
found... 12 186
found2... 1 186
found... 7 187
found... 2 188
found... 8 189
found2... 0 189
found... 6 190
found... 15 191
found... 14 192
found2... 1 192
found... 11 193
found... 8 194
found2... 7 195
found2... 11 195
found... 6 196
found... 7 197
found2... 15 198
found2... 12 198
found... 13 199
found... 1 200
found2... 7 201
found2... 1 201
found... 0 202
found... 2 203
found2... 5 204
found2... 0 204
found... 2 205
found... 3 206
found2... 4 207
found2... 6 207
found... 15 208
found... 10 209
found2... 8 210
found2... 14 210
found... 10 211
found... 15 212
found2... 14 213
found2... 9 213
found... 4 214
found... 8 215
found2... 1 215
found... 11 216
found... 7 217
found... 14 218
found2... 11 218
found... 13 219
found... 1 220
found... 12 221
found2... 8 221
found... 6 222
found... 8 223
found... 5 224
found2... 12 224
found... 0 225
found2... 12 226
found... 7 227
found2... 3 227
found... 11 228
found... 5 229
found... 4 230
found2...

In [134]:
frame_bitcodes[5]

112    10
113    10
114    10
115    10
116    10
117    10
118    10
119    10
120    10
121    10
122    10
123    10
124    10
125    10
126    10
127    10
128    10
129    10
130    10
131    10
132    10
133    10
134    10
Name:  pixel_clock, dtype: int64

In [53]:
frame_bitcodes[1049]

23450    2
23451    0
23452    0
23453    0
23454    0
23455    0
23456    0
23457    0
23458    0
23459    0
23460    0
23461    0
23462    0
23463    0
23464    0
23465    0
23466    9
23467    9
23468    9
23469    9
23470    9
23471    9
Name:  pixel_clock, dtype: int64

In [634]:
iti_bitcode = 1
looking = True
while looking is True:
    #for idx,frame in enumerate(sorted(curr_frames)):
    for frame in sorted(frame_bitcodes.keys()):

        if len([i for i in frame_bitcodes[frame] if i==iti_bitcode])>=4
            iti_frame.append(frame)
            print "found...", bitcode, frame #len(curr_frames)

224    10
225    10
226    10
227    10
228    10
229    10
230    10
231    10
232    10
233    10
234    10
235    10
236    10
237    10
238    10
239    10
240    10
241    10
242    10
243    10
244     0
245    10
Name:  pixel_clock, dtype: int64

In [ ]:
trialdict = dict()
allframes = frame_bitcodes.keys()
curr_frames = sorted(allframes[0:]) #, key=natural_keys)

for trial in ['9']: #sorted(trials.keys(), key=natural_keys): #sorted(trials.keys, key=natural_keys):
    print trial
    trialdict[trial] = dict()
    trialdict[trial]['name'] = trials[trial]['stimuli']['stimulus']
    trialdict[trial]['duration'] = trials[trial]['stim_off_times'] - trials[trial]['stim_on_times']

    first_found_frame = []
    frame_counter = 0
    
    post_iti_code = trials[str(int(trial)-1)]['iti_bitcode']
    for 
        #print bitcode # = 2
#         looking = True
#         while looking is True:
#             for idx,frame in enumerate(sorted(curr_frames)):
#                 if len([i for i in frame_bitcodes[frame] if i==bitcode])>=4:
#                     print "found...", bitcode, len(curr_frames)
#                     # Check that curr bitcode didn't start on previous frame
#                     #if bitcode in frame_bitcodes[curr_frames[idx-1]]:
#                     #     first_frame = frame-1 #(frame_counter+idx)-1
#                     #else:
#                     first_frame = frame # frame_counter+idx
#                     #print first_frame
#                     looking = False

#                 if looking==False:
#                     break

#         first_found_frame.append((bitcode, first_frame))
#         curr_frames = curr_frames[first_frame:] #curr_frames[idx:] #curr_frames[first_frame:]
#         frame_counter += first_frame
            looking = True
            while looking is True:
                #for idx,frame in enumerate(sorted(curr_frames)):
                for frame in sorted(curr_frames):
                    #print frame
                    if len([i for i in frame_bitcodes[frame] if i==bitcode])>=4:
                        print "found...", bitcode, frame #len(curr_frames)
                        # Check that curr bitcode didn't start on previous frame
                        #if bitcode in frame_bitcodes[curr_frames[idx-1]]:
                        #     first_frame = frame-1 #(frame_counter+idx)-1
                        #else:
                        #first_frame = frame # frame_counter+idx
                        #print first_frame
                        looking = False

                    if looking is False:
                        break

            first_found_frame.append((bitcode, frame)) #first_frame))
            curr_frames = curr_frames[frame:] #curr_frames[idx:] #curr_frames[first_frame:]
            frame_counter += frame #first_frame

    print (first_found_frame[-1][1] - first_found_frame[0][1])/framerate
  
    
    trialdict[trial]['stim_on_idx'] = first_found_frame[0][1]
    trialdict[trial]['stim_off_idx'] = first_found_frame[-1][1]
    
    #print first_found_frame[-1][1] - first_found_frame[0][1]

#

In [621]:
frame_bitcodes[3759]

84037     2
84038     2
84039     2
84040     2
84041     2
84042     2
84043     2
84044     2
84045     2
84046    11
84047    13
84048    13
84049    13
84050    13
84051    13
84052    13
84053    13
84054    13
84055    13
84056    13
84057    13
84058    13
Name:  pixel_clock, dtype: int64

In [592]:
4022/volumerate

89.97219407328863

In [544]:
    for trial in sorted(trialdict.keys(), key=natural_keys): #sorted(trials.keys, key=natural_keys):
        stimid = stimlist.index(trialdict[trial]['name'])
        trialdict[trial]['stimid'] = stimid
    
    stimorder = [trialdict[trial]['stimid'] for trial in sorted(trialdict.keys(), key=natural_keys)]
    with open(os.path.join(path_to_paradigm_files, 'stimorder_test.txt'),'w') as f:
        f.write('\n'.join([str(n) for n in stimorder])+'\n')
    print 
    

In [546]:
trialdict['1']

{'duration': 1983.0,
 'name': u'grating-ori-60-sf-0.500000',
 'stim_off_idx': 273,
 'stim_on_idx': 184,
 'stimid': 5}

In [542]:
stimlist = set([trialdict[trial]['name'] for trial in trialdict.keys()])
stimlist = sorted(stimlist, key=natural_keys)
stimlist

[u'grating-ori-0-sf-0.100000',
 u'grating-ori-0-sf-0.500000',
 u'grating-ori-30-sf-0.100000',
 u'grating-ori-30-sf-0.500000',
 u'grating-ori-60-sf-0.100000',
 u'grating-ori-60-sf-0.500000',
 u'grating-ori-90-sf-0.100000',
 u'grating-ori-90-sf-0.500000',
 u'grating-ori-120-sf-0.100000',
 u'grating-ori-120-sf-0.500000',
 u'grating-ori-150-sf-0.100000',
 u'grating-ori-150-sf-0.500000']

In [540]:
stimid = [s for s in sorted(stimlist, key=natural_keys)].index(trialdict[trial]['name'])
print trialdict[trial]['name']
print stimid      

grating-ori-60-sf-0.100000
4


In [578]:
first_found_frame = []
curr_frames = frame_bitcodes.keys()
frame_counter = 0

for bitcode in trials['19']['all_bitcodes']:
    #print bitcode # = 2
    looking = True
    while looking is True:
        for idx,frame in enumerate(sorted(curr_frames)):
            if len([i for i in frame_bitcodes[frame] if i==bitcode])>=4:
                #print "found...", bitcode, len(curr_frames)
                # Check that curr bitcode didn't start on previous frame
                #if bitcode in frame_bitcodes[curr_frames[idx-1]]:
                #     first_frame = frame-1 #(frame_counter+idx)-1
                #else:
                first_frame = frame # frame_counter+idx
                #print first_frame
                looking = False

            if looking==False:
                break

    first_found_frame.append((bitcode, first_frame))
    curr_frames = curr_frames[idx:] #curr_frames[first_frame:]
    frame_counter += first_frame
    
print first_found_frame[-1][1] - first_found_frame[0][1]


4241


In [577]:
first_found_frame

[(10, 4),
 (9, 195),
 (10, 202),
 (8, 202),
 (1, 213),
 (4, 216),
 (13, 228),
 (0, 229),
 (10, 241),
 (5, 243),
 (12, 261),
 (15, 469),
 (6, 477),
 (8, 477),
 (9, 481),
 (11, 483),
 (5, 485),
 (14, 489),
 (13, 495),
 (15, 495),
 (1, 498),
 (0, 503),
 (3, 527),
 (11, 538),
 (1, 722),
 (4, 736),
 (15, 744),
 (1, 750),
 (14, 762),
 (12, 775),
 (8, 780),
 (5, 786),
 (8, 796),
 (11, 803),
 (0, 996),
 (9, 1004),
 (2, 1055),
 (7, 1055),
 (4, 1066),
 (5, 1069),
 (2, 1070),
 (9, 1273),
 (8, 1274),
 (12, 1313),
 (9, 1322),
 (0, 1573),
 (11, 1582),
 (0, 1582),
 (4, 1603),
 (1, 1613),
 (7, 1618),
 (11, 1802),
 (13, 1802),
 (11, 1802),
 (3, 1803),
 (13, 1803),
 (6, 1805),
 (7, 1822),
 (5, 1828),
 (0, 1836),
 (8, 1837),
 (13, 1849),
 (2, 1856),
 (15, 1869),
 (12, 1871),
 (7, 1887),
 (11, 2068),
 (12, 2068),
 (0, 2069),
 (7, 2110),
 (8, 2112),
 (7, 2120),
 (13, 2120),
 (11, 2125),
 (4, 2151),
 (1, 2156),
 (15, 2340),
 (2, 2376),
 (10, 2385),
 (14, 2388),
 (9, 2398),
 (11, 2399),
 (7, 2413),
 (12, 242

In [516]:
# found_bitcodes = [match[0] for match in first_found_frame]
# mw_bitcodes = trials['1']['all_bitcodes']
# [idx for idx,(mw,ard) in enumerate(zip(found_bitcodes, mw_bitcodes)) if not mw==ard]